In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('scraped_data.csv')

In [4]:
# drops duplicates by finding matches from the combined ['company','title'] columns
df = df.drop_duplicates(subset=['company', 'title'])

In [5]:
# create ['js_count','python_count','ruby_count','java_count'] columns with counts of each showing up in job descriptions
descriptions = df['description']
df['js_count'] = descriptions.str.count(r'[jJ]ava[sS]cript\b')
df['python_count'] = descriptions.str.count(r'[Pp]ython\b')
df['ruby_count'] = descriptions.str.count(r'[Rr]uby\b')
df['java_count'] = descriptions.str.count(r'[Jj]ava\b')
df[['js_count', 'python_count', 'ruby_count', 'java_count']] = \
    df[['js_count', 'python_count', 'ruby_count', 'java_count']].fillna(0).astype(int)

In [6]:
# df[['search_terms', 'title', 'company', 'js_count', 'python_count', 'ruby_count', 'java_count']].sort_values(by=['ruby_count'], ascending=False).head(50)

In [7]:
# recategorizes 'search_terms' for each posting based on highest tech count in description
highest_tcount = df[['js_count', 'python_count', 'ruby_count', 'java_count']].idxmax(axis=1)
mapper = {
    'js_count': 'javascript developer', 
    'python_count': 'python developer', 
    'ruby_count': 'ruby developer', 
    'java_count': 'java developer'
}
df['search_terms'] = highest_tcount.map(lambda x: mapper[x])

In [8]:
def format_hourly(rate):
    '''Returns annual salaries converted from hourly rates'''
    return rate * 2000 if rate > 0 and rate < 1000 else rate

In [9]:
# creates the 'formatted_sal' column by finding and formatting high range of scraped salary data
sal_cap_groups = df['salary'].str.extract(r'\d\d\d?,\d\d\d\s-\s\$(\d\d\d?,\d\d\d)|\d\d\s-\s\$(\d\d)')
combined_sal_groups = sal_cap_groups[0].combine_first(sal_cap_groups[1]).str.replace(',', '')
df['formatted_sal'] = combined_sal_groups.fillna(0).astype(int).map(format_hourly).replace(0, np.nan).astype('Int64')
# df_dd['formatted_sal'] = combined.fillna(0).astype(int).map(lambda num: num * 2000 if num > 0 and num < 1000 else num).replace(0, np.nan)

In [10]:
# groups the data by city and technology and returns average salaries for each
grouped = df.groupby(['search_loc', 'search_terms'])
market_sals = grouped['formatted_sal'].mean().sort_values(ascending=False).astype(int)

In [18]:
# counts the overall metrics for position counts by tech and amount of total positions per market
# also shows top salaries in the database
pcount_by_tech = df['search_terms'].value_counts()
pcount_by_loc = df['search_loc'].value_counts()
top_salaries = df[df['formatted_sal'] > 0].sort_values(by='formatted_sal', ascending=False)

Washington, DC       2076
San Francisco, CA    1534
New York, NY         1530
Seattle, WA          1289
Boston, MA           1274
Austin, TX            868
Charlotte, NC         388
Name: search_loc, dtype: int64

In [12]:
# finding salaried ruby jobs in washington
# pcount_by_loc = df[['search_loc','formatted_sal','search_terms']].value_counts()
# dc_counts = pcount_by_loc['Washington, DC'].reset_index()
# dc_counts[pcount_by_loc['Washington, DC'].reset_index()['search_terms'] == 'ruby developer']

In [13]:
# creates new dataframe with position counts per market, position % breakdown per market, & overall marketshare of tech by market
pos_metrics_mkt = pd.DataFrame(columns=['pos_counts_mkt', 'pos_pcts_mkt', 'pos_overall_mkt_pct'])
pos_metrics_mkt['pos_pcts_mkt'] = round(df.groupby(['search_loc'])['search_terms'].value_counts(normalize=True) * 100, 2)
pos_metrics_mkt['pos_counts_mkt'] = df.groupby(['search_loc'])['search_terms'].value_counts()
pos_metrics_mkt['pos_overall_mkt_pct'] = round(pos_metrics_mkt['pos_counts_mkt'] / pos_metrics_mkt['pos_counts_mkt'].sum() * 100, 2)
pos_metrics_mkt.reset_index(inplace=True)

In [14]:
pos_metrics_mkt.to_csv('seed_pos_metrics_mkt.csv', encoding='utf-8-sig')
market_sals.to_csv('seed_market_sals.csv', encoding='utf-8-sig')
df.to_csv('seed_positions.csv',index=False,encoding='utf-8-sig')